<a href="https://colab.research.google.com/github/Satwikram/NLP-Using-Deep-Learning/blob/master/Twitter%20Sentiment%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Author: Satwik Ram K

## Twitter Sentiment Analysis

###Connecting to Kaggle for fast computation

In [ ]:
from google.colab import files

files.upload()


! mkdir ~/.kaggle


! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


###Dowloading the dataset

In [ ]:
! kaggle datasets download -d imrandude/twitter-sentiment-analysis

  0% 0.00/3.67M [00:00<?, ?B/s]
100% 3.67M/3.67M [00:00<00:00, 58.0MB/s]


#### Unzipping the dataset

In [ ]:
! unzip /content/twitter-sentiment-analysis.zip

Archive:  /content/twitter-sentiment-analysis.zip
  inflating: train.csv               


### Importing Dependencies

In [ ]:
import numpy as np
import pandas as pd
import re
import tensorflow as tf
from tensorflow import keras
import math 
from bs4 import BeautifulSoup

In [ ]:
tf.__version__

'2.2.0'

### Loading Dataset

In [ ]:
dataset = pd.read_csv('/content/train.csv', engine = 'python', encoding = 'latin1')

#### Dataset info
SentimentText - text of the twit

0 - negative

1 - positive

In [ ]:
dataset.head()

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...


In [ ]:
dataset['SentimentText'][3]

"          .. Omgaga. Im sooo  im gunna CRy. I've been at this dentist since 11.. I was suposed 2 just get a crown put on (30mins)..."

In [ ]:
dataset.drop('ItemID', axis = 1, inplace = True)

In [ ]:
dataset.tail()

,Sentiment,SentimentText
99984,0,@Cupcake seems like a repeating problem hop...
99985,1,@cupcake__ arrrr we both replied to each other...
99986,0,@CuPcAkE_2120 ya i thought so
99987,1,@Cupcake_Dollie Yes. Yes. I'm glad you had mor...
99988,1,@cupcake_kayla haha yes you do


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99989 entries, 0 to 99988
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Sentiment      99989 non-null  int64 
 1   SentimentText  99989 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.5+ MB


In [ ]:
dataset.isna().sum()

Sentiment        0
SentimentText    0
dtype: int64

In [ ]:
dataset['Sentiment'].unique()

array([0, 1])

### Cleaning the Tweets using Regular Expression

In [ ]:
def clean_tweet(tweet):
  tweet = BeautifulSoup(tweet, 'lxml').get_text()
  tweet = re.sub(r"@[A-Za-z0-9]+", "", tweet)
  tweet = re.sub(r"https?://[A-Za-z0-9./]", "", tweet)
  tweet = re.sub(r"[^A-Za-z.?!]", "", tweet)
  tweet = re.sub(r"[A-Za-z0-9]!", "", tweet)
  tweet = re.sub(r"! ", "", tweet)
  tweet = re.sub(r". ", "", tweet)
  tweet = re.sub(r" +", '', tweet)
  tweet = re.sub(r"\.+$", '', tweet)
  return tweet


In [ ]:
data_clean = [clean_tweet(tweet) for tweet in dataset.SentimentText]

In [ ]:
data_clean

['issosadformyAPLfriend',
 'ImissedtheNewMoontrailer',
 'omgitsalreadyO',
 '..Omgaga.ImsoooimgunnaCRy.Ivebeenatthisdentistsince..Iwassuposedjustgetacrownputonmins',
 'ithinkmibfischeatingonm!!TT',
 'orijustworrytoomuch?',
 'JuuuuuuuuuuuuuuuuussssstChilli!',
 'SunnyAgainWorkTomorrowTVTonight',
 'handedinmyuniformtoday.imissyoualready',
 'hmmmm....iwonderhowshemynumber',
 'Imustthinkaboutpositive',
 'thankstoallthehatersupinmyfaceallda',
 'thisweekendhassuckedsofar',
 'jbisntshowinginaustraliaanymor',
 'okthatsityouwin',
 'Thisisthewayifeelrightnow',
 'awhheman....Imcompletelyuselessrtnow.FunnyallIcandoistwitter.yloc.meHX',
 'Feelingstrangelyfine.NowImgonnagolistentosomeSemisonictocelebrate',
 'HUGErollofthunderjustnow...SOscar!!!',
 'Ijustcutmybeardoff.Itsonlybeengrowingforwelloverayear.Imgonnastartitover.ishappyinthemeantime',
 'VerysadaboutIran',
 'womppppwompp',
 'Youretheonlyonewhocanseethiscausenooneelseisfollowingmethisisforyoubecauseyoureprettyawesome',
 'Sadlevelis.Iwaswritingam